**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [18]:
!pip install sk-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout
from tensorflow import reduce_max

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function `act` allows the Agent to determine the following interaction with the Environment. If we already know a good policy of action, then `learned_act` will return an action which leads to a good reward. However, if our current "best policy" (given by `learned_act`) is not yet optimal,  without `epsilon`, we would not consider any other possible action and keep with our non optimal policy. With `epsilon`, et each new action to be taken, we allow the possibility (often quite small) to try another random action. If this new action happen to lead to good reward, then we can store this information and adapt our policy (via `reinforce` method) in order to obtain better reward than our current policy the next time we will face the same current state. To summarize, `epsilon` allows exploration of actions.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:                    #go up
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:                    #go down
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:                    #go right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:                    #go left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        #eat the content of the new position and get the corresponding reward
        reward = self.board[self.x, self.y]   
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # the shape of a state is (5,5,2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

### Parameters

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=31 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array `board` saves the global state of the game, with the current content of the cases (eather a piece of cheese = a bonus, a poisonous cell = a malus or 0 if the rat has already been through the cell before the current time). It is updated along with the movements of the rat (the agent).  

The array `position` is used to build the current visual field of the rat : if the rat is not close to any border, then his visual field is full, an array of size $5\times 5$ full of $0$. However, if he gets close from a border, he will see the border, which is represented by values $-1$ in its visual field. We extract this visual field (second array of the variable `state`) from the variable `position`, around the current position (`self.x`, `self.y`) of the rat. When an action is taken by the agent, the variable `position` also stores the previous location of the agent (the new one is stored with (`self.x`, `self.y`)) by placing a $1$ to the current `state` to the previous location.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]
        

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        
        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)


        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        
        print("Epoch {:03d}/{:03d}".format(e+1, epochs))

        print("Win/lose count {}/{}. Average score ({})\n"
              .format(win, lose, score/(1+e)))
        
        
    print('Final score: '+str(score/epochs))


In [26]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Epoch 001/021
Win/lose count 11.5/18.0. Average score (-6.5)

Epoch 002/021
Win/lose count 12.0/15.0. Average score (-4.75)

Epoch 003/021
Win/lose count 7.5/16.0. Average score (-6.0)

Epoch 004/021
Win/lose count 11.0/14.0. Average score (-5.25)

Epoch 005/021
Win/lose count 11.0/11.0. Average score (-4.2)

Epoch 006/021
Win/lose count 9.0/11.0. Average score (-3.8333333333333335)

Epoch 007/021
Win/lose count 8.0/16.0. Average score (-4.428571428571429)

Epoch 008/021
Win/lose count 9.0/12.0. Average score (-4.25)

Epoch 009/021
Win/lose count 11.5/10.0. Average score (-3.611111111111111)

Epoch 010/021
Win/lose count 5.0/15.0. Average score (-4.25)

Epoch 011/021
Win/lose count 12.5/16.0. Average score (-4.181818181818182)

Epoch 012/021
Win/lose count 9.5/11.0. Average score (-3.9583333333333335)

Epoch 013/021
Win/lose count 9.5/17.0. Average score (-4.230769230769231)

Epoch 014/021
Win/lose count 8.5/16.0. Average score (-4.464285714285714)

Epoch 015/021
Win/lose count 11.0/9.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***

Let us recall that :

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Then, we can write :

\begin{align*}
Q^\pi(s,a) & =E[\sum_{t=0}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
& = E[r(s,a) + \sum_{t=1}^{+\infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]  \\
& = r(s,a) + \gamma E[ \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]  \\
& = r(s,a) + \gamma E[E[ \sum_{t=0}^{+\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_{1}=s',a_{1}=a']|s_{0}=s,a_{0}=a]  \\
& = r(s,a) + \gamma E[Q^\pi(s', a')|s_{0}=s,a_{0}=a]  \\
& = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

Consequently, if we recall that :

$$ Q^*(s,a)  = \max_{\pi} Q^\pi(s,a)$$

We obtain :

\begin{align*}
Q^*(s,a) &= \max_{\pi} E_{(s',a')|(s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\
&= r(s,a) + \gamma \max_{\pi} \sum_{(a', s')} \pi(a'|s')p(s'|s,a)Q^\pi(s',a') \\
&= r(s,a) + \gamma E_{s' \sim p(. |s,a)}[\max_{\pi}\sum_{a'} \pi(a'|s')Q^\pi(s',a')]
\end{align*}

We notice that this quantity is maximal when $\pi(a'|s') = 1$ for $a' = \text{arg} \max_{a} Q^*(s', a)$ and $\pi(a'|s') = 0$ otherwise.
Then, we obtain :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}



Using such a loss guaranties that $\mathcal{L}(\theta) \geq 0$ and $\mathcal{L}(\theta^*) = 0$, so it seems plausible to use it.






***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        ind = np.random.randint(0, len(self.memory))    #returns integer in [0, len(self.memory)-1]
        return self.memory[ind]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

### DQN Agent


***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # We suppose we have a trained model that predict the next reward for any action taken from state s
        Q_values = self.model.predict(s[np.newaxis,:])[0]

        # We simply take the action that maximizes the reward
        return np.argmax(Q_values)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #build an element of batch from previous steps
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i] = s

            #access the current value of Q for all the actions
            target_q[i] = self.model.predict(s[np.newaxis,:])[0]

            #there are two possibilities : 
            
            #either the game is over after this action          
            if game_over:
                #in this situation, we don't have new information : we keep the reward with the same value
                target_q[i, a] = r

            #either the game keeps going : the target value of q is obtained
            # with the asymptotic value obtained with Bellman equation, for this given action
            else:
                target_q[i, a] = r + self.discount*max(self.model.predict(n_s[np.newaxis,:])[0])
                
        # We can now update the weight and the loss of the model.
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model
      

### Fully connected model

In [0]:
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32,activation ='relu')) #simple one hidden layer network
        model.add(Dropout(0.2))
        model.add(Dense(4, activation='linear'))
        model.summary()
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model

We have tried to run this model with various different shapes of the neural network, however we obtained the best results with quite narrow layers (only 32 neurons on the hidden layer). We changed the solver, Adam performs better in this situation, and we also decresed the learning rate, in orde to try to avoid oscillating loss. It requires to run more epochs though. We finally obtain decent score.

In [97]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train30.mp4'))

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_29 (Flatten)         (None, 50)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 32)                1632      
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 4)                 132       
Total params: 1,764
Trainable params: 1,764
Non-trainable params: 0
_________________________________________________________________
Epoch 001/031 | Loss 0.0206 | Win/lose count 6.0/3.0 (3.0)
Epoch 002/031 | Loss 0.0114 | Win/lose count 0/5.0 (-5.0)
Epoch 003/031 | Loss 0.0099 | Win/lose count 5.0/4.0 (1.0)
Epoch 004/031 | Loss 0.0102 | Win/lose count 6.0/1.0 (5.0)
Epoch 005/031 

***
***
### Convolutional Network

__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(64, (2,2), activation='relu', input_shape=( 5, 5, self.n_state)))
        model.add(MaxPooling2D(pool_size=2))
        model.add(Conv2D(32, (2,2), activation='relu'))
        model.add(Flatten())
        model.add(Dropout(0.2))
        model.add(Dense(4, activation='linear'))
        model.summary()
        
        model.compile(Adam(lr=lr), "mse")
        self.model = model

In [99]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train30.mp4'))

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 4, 4, 64)          576       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 1, 1, 32)          8224      
_________________________________________________________________
flatten_31 (Flatten)         (None, 32)                0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 4)                 132       
Total params: 8,932
Trainable params: 8,932
Non-trainable params: 0
___________________________________________________

We tries a convolutional network, we added some dropout and some pooling layer. We also reduced even more the learning rate, as there are way more parameters to train (roughly 5 times more). The algorithm performs quite good too. 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [101]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')


Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 4, 4, 64)          576       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 1, 1, 32)          8224      
_________________________________________________________________
flatten_34 (Flatten)         (None, 32)                0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 4)                 132       
Total params: 8,932
Trainable params: 8,932
Non-trainable params: 0
___________________________________________________

In [103]:
HTML(display_videos('fc_test10.mp4'))

In [104]:
HTML(display_videos('cnn_test10.mp4'))

In [105]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_t05_test')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_t05_test')


Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 4, 4, 64)          576       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 1, 1, 32)          8224      
_________________________________________________________________
flatten_36 (Flatten)         (None, 32)                0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_58 (Dense)             (None, 4)                 132       
Total params: 8,932
Trainable params: 8,932
Non-trainable params: 0
___________________________________________________

In [109]:
HTML(display_videos('fc_t05_test10.mp4'))

In [110]:
HTML(display_videos('cnn_t05_test10.mp4'))

In [106]:
env = Environment(grid_size=size, max_time=T,temperature=0.7)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_t07_test')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_t07_test')


Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 4, 4, 64)          576       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 1, 1, 32)          8224      
_________________________________________________________________
flatten_38 (Flatten)         (None, 32)                0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_61 (Dense)             (None, 4)                 132       
Total params: 8,932
Trainable params: 8,932
Non-trainable params: 0
___________________________________________________

In [107]:
HTML(display_videos('fc_t07_test10.mp4'))

In [108]:
HTML(display_videos('cnn_t07_test10.mp4'))

We notice that the agent does not explore much of the environment. When it is in an empty neighbourhood (because it has visited everything), it tends to stay in such a restricted area, doing some loops. However, when we augment the temperature, which corresponds to a denser environment, the rat tends to explore a bit more.

We could think to a strategy that encourages exploration with a malus for going through places that have already been explored.

***

### Exploration

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, eps_decrease=0.9, prefix=''):
    

    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        #decrease value of epsilon
        agent.set_epsilon(agent.epsilon*eps_decrease)
        print("Value of epsilon : ", agent.epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

            
            

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, malus_value=0.2):

        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.malus_value = malus_value

        #board on which one plays
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))


        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:                    #go up
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:                    #go down
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:                    #go right
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:                    #go left
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1



        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = self.malus_value

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0

        
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # the shape of a state is (5,5,3) now
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),
                        axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [115]:
epochs_train =51

# lower learning rate and bigger batch size => more training

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, malus_value=.1)

# We use a particularly high epsilon as it is decreasing with time,
# and we want large exploration at start.
agent = DQN_CNN(size, lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 4, 4, 64)          832       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 1, 1, 32)          8224      
_________________________________________________________________
flatten_42 (Flatten)         (None, 32)                0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_66 (Dense)             (None, 4)                 132       
Total params: 9,188
Trainable params: 9,188
Non-trainable params: 0
___________________________________________________

The scores we obtain from the training step appear quite low as they include the malus from going through already explored places. Let us see how everything sums up durnig test phase :

In [116]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Epoch 001/011
Win/lose count 17.0/6.0. Average score (11.0)

Epoch 002/011
Win/lose count 22.0/7.0. Average score (13.0)

Epoch 003/011
Win/lose count 22.0/3.0. Average score (15.0)

Epoch 004/011
Win/lose count 20.5/8.0. Average score (14.375)

Epoch 005/011
Win/lose count 24.5/6.0. Average score (15.2)

Epoch 006/011
Win/lose count 20.5/4.0. Average score (15.416666666666666)

Epoch 007/011
Win/lose count 27.5/6.0. Average score (16.285714285714285)

Epoch 008/011
Win/lose count 14.0/4.0. Average score (15.5)

Epoch 009/011
Win/lose count 22.0/5.0. Average score (15.666666666666666)

Epoch 010/011
Win/lose count 24.0/5.0. Average score (16.0)

Epoch 011/011
Win/lose count 19.5/4.0. Average score (15.954545454545455)

Final score: 15.954545454545455


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***